# Correlation Between SDO and GOES-14 Data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.time
import astropy.units as u
from astropy.visualization import quantity_support, time_support
from sunpy.net import Fido
from sunpy.net import attrs as a
import sys
from scipy.signal import savgol_filter
from scipy import stats
import pandas as pd
import cdflib
import dateutil
import dask.dataframe as dd
from dask.distributed import Client
from scipy.signal import savgol_filter
from scipy import stats
import warnings
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
import netCDF4 #as nc
from scipy.io import netcdf
from netCDF4 import Dataset as nc_dataset
import glob
import xarray as xr
#from sscws.sscws import SscWs
import warnings
warnings.filterwarnings("ignore")
#%matplotlib widget
#from yellowbrick.regressor import CooksDistance

# Read Epead and SDO

In [2]:
# SDO
df_sdo = dd.read_parquet("/efs/sdoradbelt/data/sdo_headers", columns=['NSPIKES','WAVELNTH','GAEZ_OBS'])
df_sdo = df_sdo.set_index(df_sdo.index, sorted=True) # solve the sorting issue

In [3]:
year = 2018; month = 1
df_goes = xr.open_dataset(glob.glob('/efs/sdoradbelt/data/raw_data/goes14_epead_science/g14_epead_e13ew_1m_{}{}01'.format(str(year),str(month).zfill(2)) + '*.nc')[0]).to_dataframe()
df_goes = df_goes.set_index('time_tag')
i = 0
for year in [2018,2019]:
    for month in range(1,13):
        if i>0:
            df2 = xr.open_dataset(glob.glob('/efs/sdoradbelt/data/raw_data/goes14_epead_science/g14_epead_e13ew_1m_{}{}01'.format(str(year),str(month).zfill(2)) + '*.nc')[0]).to_dataframe()
            df2 = df2.set_index('time_tag')
            df_goes = df_goes.append(df2)
        i += 1

# Get Equator - Non Equator Correlations

In [5]:
year_start = 2018; year_end = 2019; month_start = 1; month_end = 12; day_start = 1; day_end = 30
start_d = "{}-{}-{}".format(year_start,str(month_start).zfill(2),str(day_start).zfill(2)); end_d = "{}-{}-{}".format(year_end,str(month_end).zfill(2),str(day_end).zfill(2)) 
cadence = '{}min'.format(1) #change to 15 minutes
instruments = ['E1W_DTC_FLUX','E1E_DTC_FLUX','E2W_DTC_FLUX','E2E_DTC_FLUX','E1W_COR_FLUX','E1E_COR_FLUX','E2W_COR_FLUX','E2E_COR_FLUX']
for inst in instruments:
    print()
    print(inst)
    wavelengths = [94,131,171,193,211,304,335]
    for wl in wavelengths:
        z_sdo = df_sdo[df_sdo['WAVELNTH'] == wl]['GAEZ_OBS'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
        nspikes = df_sdo[df_sdo['WAVELNTH'] == wl]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
        cor_flux = df_goes[inst].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().interpolate()
        cor_flux[cor_flux<0] = 0
        # Pick Equator
        nspikes_eq = []; nspikes_non_eq = []
        e_flux_eq = []; e_flux_non_eq = []
        for i in range(0,len(z_sdo)):
            if z_sdo[i] >= -2*10**6 and z_sdo[i] <= 2*10**6:
                nspikes_eq.append(nspikes[i])
                e_flux_eq.append(cor_flux[i])
            else:
                nspikes_non_eq.append(nspikes[i])
                e_flux_non_eq.append(cor_flux[i])
        # Print Correlations
        print('WL',wl)
        print('Equator Pearson/Spearman Correlation')
        print('r = ',stats.pearsonr(nspikes_eq,e_flux_eq)[0],' / rho = ',stats.spearmanr(nspikes_eq,e_flux_eq)[0])
        print('Non-Equator Pearson/Spearman Correlation')
        print('r = ',stats.pearsonr(nspikes_non_eq,e_flux_non_eq)[0],' / rho = ',stats.spearmanr(nspikes_non_eq,e_flux_non_eq)[0])
        print('Combined Pearson/Spearman Correlation')
        print('r = ',stats.pearsonr(nspikes,cor_flux)[0],' / rho = ',stats.spearmanr(nspikes,cor_flux)[0])
        print()


E1W_DTC_FLUX
WL 94
Equator Pearson/Spearman Correlation
r =  0.364836282607857  / rho =  0.5286898436037822
Non-Equator Pearson/Spearman Correlation
r =  0.3390388770784085  / rho =  0.38321740859569475
Combined Pearson/Spearman Correlation
r =  0.339784884892583  / rho =  0.38684254903728665

WL 131
Equator Pearson/Spearman Correlation
r =  0.3706925942032124  / rho =  0.5341883444532916
Non-Equator Pearson/Spearman Correlation
r =  0.34443369476244734  / rho =  0.38538080851535217
Combined Pearson/Spearman Correlation
r =  0.34520870421663635  / rho =  0.38910310864364317

WL 171
Equator Pearson/Spearman Correlation
r =  0.21660086905079684  / rho =  0.4317527314799798
Non-Equator Pearson/Spearman Correlation
r =  0.23883340165506156  / rho =  0.34988323040338004
Combined Pearson/Spearman Correlation
r =  0.2371165692931505  / rho =  0.35149030001150955

WL 193
Equator Pearson/Spearman Correlation
r =  0.21863694101174416  / rho =  0.4352403746138262
Non-Equator Pearson/Spearman Cor

# Tests

In [ ]:
year_start = 2019; year_end = 2019
month_start = 1; month_end = 1
day_start = 1; day_end = 30
start_d = "{}-{}-{}".format(year_start,str(month_start).zfill(2),str(day_start).zfill(2)); end_d = "{}-{}-{}".format(year_end,str(month_end).zfill(2),str(day_end).zfill(2)) 
cadence = '{}min'.format(30) #change to 15 minutes
cor_flux = df_goes['E1E_COR_FLUX'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().interpolate()
nspikes = df_sdo[df_sdo['WAVELNTH'] == 94]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
cor_flux[cor_flux<=0] = 0.1
nspikes[nspikes<=0] = 0.1
plt.plot(cor_flux)
plt.show()
plt.plot(nspikes)
plt.show()
print(stats.pearsonr(nspikes,cor_flux)[0])
print(stats.spearmanr(nspikes,cor_flux)[0])

In [ ]:
print(stats.pearsonr(np.log(nspikes),np.log(cor_flux))[0])
print(stats.spearmanr(np.log(nspikes),np.log(cor_flux))[0])

In [ ]:
e_flux_kV = [40,75,150,275,475]
for telescope in range(1,10): 
    print('Telescope: {}'.format(telescope))
    for flux in  e_flux_kV: # choose from 1 to 5
        wavelengths = [94,131,171,193,211,304,335]
        wl_p_cor_eq = []; wl_s_cor_eq = []; wl_p_cor_neq = []; wl_s_cor_neq = []; wl_p_cor_all = []; wl_s_cor_all = []
        for wl in wavelengths:
            print(wl)
            # GOES (Select Telescope, Electron Flux and Parameter)
            df_goes = dd.read_parquet("/efs/sdoradbelt/data/goes_data/maged/goes_telescope{}_keV{}.parquet".format(telescope,flux))
            # Get headers in all wavelengths along with GOES-14 data. Convert to int32 for less memory. Select time span, cadence and interpolate in case of NaNs.
            year_start = 2018; year_end = 2019
            month_start = 1; month_end = 12
            day_start = 1; day_end = 30
            start_d = "{}-{}-{}".format(year_start,str(month_start).zfill(2),str(day_start).zfill(2)); end_d = "{}-{}-{}".format(year_end,str(month_end).zfill(2),str(day_end).zfill(2)) 
            cadence = '{}min'.format(1) #change to 15 minutes
            z_sdo = df_sdo[df_sdo['WAVELNTH'] == wl]['GAEZ_OBS'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
            nspikes = df_sdo[df_sdo['WAVELNTH'] == wl]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
            cor_flux = df_goes['cor_flux'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
            cor_flux[cor_flux<0] = 0
            # Pick Equator
            nspikes_eq = []; nspikes_non_eq = []
            e_flux_eq = []; e_flux_non_eq = []
            for i in range(0,len(z_sdo)):
                if z_sdo[i] >= -2*10**6 and z_sdo[i] <= 2*10**6:
                    nspikes_eq.append(nspikes[i])
                    e_flux_eq.append(cor_flux[i])
                else:
                    nspikes_non_eq.append(nspikes[i])
                    e_flux_non_eq.append(cor_flux[i])
            wl_p_cor_eq.append(stats.pearsonr(nspikes_eq,e_flux_eq)[0])
            wl_s_cor_eq.append(stats.spearmanr(nspikes_eq,e_flux_eq)[0])
            wl_p_cor_neq.append(stats.pearsonr(nspikes_non_eq,e_flux_non_eq)[0])
            wl_s_cor_neq.append(stats.spearmanr(nspikes_non_eq,e_flux_non_eq)[0])
            wl_p_cor_all.append(stats.pearsonr(nspikes,cor_flux)[0])
            wl_s_cor_all.append(stats.spearmanr(nspikes,cor_flux)[0])
            #if float(stats.pearsonr(nspikes_eq,e_flux_eq)[0]) > float(0.75):
                #print('Good Correlation ({}) at {}/{}'.format(float(stats.pearsonr(nspikes_eq,e_flux_eq)[0]),wl,flux))

        # Print Correlations
        print('FLUX',flux)
        print('Equator Pearson/Spearman Correlation')
        print('r = ',np.mean(wl_p_cor_eq),'+-',np.std(wl_p_cor_eq),' / rho = ',np.mean(wl_s_cor_eq),'+-',np.std(wl_s_cor_eq))
        print('Non-Equator Pearson/Spearman Correlation')
        print('r = ',np.mean(wl_p_cor_neq),'+-',np.std(wl_p_cor_neq),' / rho = ',np.mean(wl_s_cor_neq),'+-',np.std(wl_s_cor_neq))
        print('Combined Pearson/Spearman Correlation')
        print('r = ',np.mean(wl_p_cor_all),'+-',np.std(wl_p_cor_all),' / rho = ',np.mean(wl_s_cor_all),'+-',np.std(wl_s_cor_all))
        print()

# Heatmap

In [ ]:
# GOES (Select Telescope, Electron Flux and Parameter)
df_goes = dd.read_parquet("/efs/sdoradbelt/data/goes_data/maged/goes_telescope{}_keV{}.parquet".format(3,40))
# Get headers in all wavelengths along with GOES-14 data. Convert to int32 for less memory. Select time span, cadence and interpolate in case of NaNs.
year_start = 2018; year_end = 2019
month_start = 1; month_end = 12
day_start = 1; day_end = 30
start_d = "{}-{}-{}".format(year_start,str(month_start).zfill(2),str(day_start).zfill(2)); end_d = "{}-{}-{}".format(year_end,str(month_end).zfill(2),str(day_end).zfill(2)) 
cadence = '{}min'.format(1) #change to 15 minutes
z_sdo = df_sdo[df_sdo['WAVELNTH'] == 304]['GAEZ_OBS'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes = df_sdo[df_sdo['WAVELNTH'] == 304]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
cor_flux = df_goes['cor_flux'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
cor_flux[cor_flux<0] = 0.1

In [ ]:
# Pick Equator
nspikes_eq = []; cor_flux_eq = []; 
for i in range(0,len(z_sdo)):
    if z_sdo[i] >= -2*10**6 and z_sdo[i] <= 2*10**6:
        nspikes_eq.append(nspikes[i])
        cor_flux_eq.append(cor_flux[i])

In [ ]:
# Getting rid of small values
idx = np.where(nspikes < 3000)[0]
nspikes_small = np.array(nspikes)[idx]
cor_flux_small = np.array(cor_flux)[idx]
nspikes = np.delete(np.array(nspikes),idx)
cor_flux = np.delete(np.array(cor_flux),idx)

In [ ]:
# Getting rid of small values FOR EQUATOR
idx_eq = np.where(nspikes < 3000)[0]
#nspikes_eq_small = np.array(nspikes_eq)[idx_eq]
#cor_flux_eq_small = np.array(cor_flux_eq)[idx]
nspikes_eq = np.delete(np.array(nspikes_eq),idx_eq)
cor_flux_eq = np.delete(np.array(cor_flux_eq),idx_eq)

In [ ]:
nspikes_log = np.log(nspikes)
cor_flux_log = np.log(cor_flux)
print(stats.pearsonr(nspikes,cor_flux)[0])
print(stats.pearsonr(nspikes_log,cor_flux_log)[0])
print(stats.spearmanr(nspikes,cor_flux)[0])
print(stats.spearmanr(nspikes_log,cor_flux_log)[0])

In [ ]:
nspikes_eq_log = np.log(nspikes_eq)
cor_flux_eq_log = np.log(cor_flux_eq)
print(stats.pearsonr(nspikes_eq,cor_flux_eq)[0])
print(stats.pearsonr(nspikes_eq_log,cor_flux_eq_log)[0])
print(stats.spearmanr(nspikes_eq,cor_flux_eq)[0])
print(stats.spearmanr(nspikes_eq_log,cor_flux_eq_log)[0])

In [ ]:
plt.hist2d(nspikes, cor_flux, bins=4000)
plt.show()

In [ ]:
# Heatmap for best possible wavelength/keV/year
plt.style.use('bmh')
figure, axis = plt.subplots(1, 2, figsize=(14,5))
figure.suptitle('NSPIKES vs GOES Electrons', fontsize=16,y=0.975)
#
axis[0].hist2d(nspikes, cor_flux, bins=4000, cmap=plt.cm.OrRd)
axis[0].set_xlim(3000,0.5*1e5)
axis[0].set_ylim(0,0.5*1e5)
axis[0].set_xlabel('NSPIKES',size=15,labelpad=10)
axis[0].set_ylabel('GOES Electrons',size=15,labelpad=10)
m, b = np.polyfit(nspikes, cor_flux, 1)
x = np.linspace(3000,0.5*1e5,50)
axis[0].plot(x, m*x+b-11000,linewidth=1,label='Best Fit')
axis[0].legend(loc='lower right',fontsize=17.5)
#
axis[1].hist2d(nspikes_log, cor_flux_log, bins=300, cmap=plt.cm.OrRd)
axis[1].set_xlim(8,13)
axis[1].set_ylim(8,13)
axis[1].set_xlabel('log(NSPIKES)',size=15,labelpad=10)
axis[1].set_ylabel('log(GOES Electrons)',size=15,labelpad=10)
m, b = np.polyfit(nspikes_log, cor_flux_log, 1)
x = np.linspace(8,13,50)
axis[1].plot(x, x-0.4,linewidth=1)

plt.savefig('Heatmaps.png')
plt.show()


In [ ]:
plt.style.use('bmh')
figure, axis = plt.subplots(1, 2, figsize=(14,5))
figure.suptitle('NSPIKES vs GOES Electrons at Equator', fontsize=16,y=0.975)

axis[0].hist2d(nspikes_eq, cor_flux_eq, bins=4000, cmap=plt.cm.OrRd)
axis[0].set_xlim(3000,0.5*1e5)
axis[0].set_ylim(0,0.5*1e5)
axis[0].set_xlabel('Equator NSPIKES',size=15,labelpad=10)
axis[0].set_ylabel('Equator GOES Electrons',size=15,labelpad=10)
m, b = np.polyfit(nspikes_log, cor_flux_log, 1)
x = np.linspace(3000,0.5*1e5,50)
axis[0].plot(x, m*x+b,linewidth=1,label='Best Fit')
axis[0].legend(loc='lower right',fontsize=17.5)
#
axis[1].hist2d(nspikes_eq_log, cor_flux_eq_log, bins=300, cmap=plt.cm.OrRd)
axis[1].set_xlim(8,13)
axis[1].set_ylim(8,13)
axis[1].set_xlabel('log(Equator NSPIKES)',size=15,labelpad=10)
axis[1].set_ylabel('log(Equator GOES Electrons)',size=15,labelpad=10)
m, b = np.polyfit(nspikes_log, cor_flux_log, 1)
x = np.linspace(8,13,50)
axis[1].plot(x, x-0.45,linewidth=1)

plt.savefig('Heatmaps_Equator.png')
plt.show()

In [ ]:
plt.style.use('bmh')
figure, axis = plt.subplots(1, 2, figsize=(14,5))
figure.suptitle('Spread of NSPIKES (60 Bins)', fontsize=16,y=0.95)

axis[0].hist(nspikes,bins=60)
axis[0].set_xlabel('NSPIKES',size=15,labelpad=10)
axis[0].set_ylabel('Number of Points',size=15,labelpad=10)
axis[0].set_facecolor('white')
#
axis[1].hist(nspikes_log,bins=60)
axis[1].set_xlabel('log(NSPIKES)',size=15,labelpad=10)
axis[1].set_facecolor('white')
#axis[1].set_ylabel('Number of Points',size=15,labelpad=10)

plt.savefig('NSPIKES_Histogram.png')
plt.show()

In [ ]:
# what are these low values? are they real or not? exclude them and check what they are. get some more depth in it. 

# Log for high energies

In [ ]:
# Try doing log of both for low energies

# Try Epead

# Equator Data Picking Plot

In [ ]:
   
# Get headers in all wavelengths along with GOES-14 data. Convert to int32 for less memory. Select time span, cadence and interpolate in case of NaNs.
year_start = 2019; year_end = 2019
month_start = 10; month_end = 10
day_start = 15; day_end = 15
start_d = "{}-{}-{}".format(year_start,str(month_start).zfill(2),str(day_start).zfill(2)); end_d = "{}-{}-{}".format(year_end,str(month_end).zfill(2),str(day_end).zfill(2)) 
cadence = '{}min'.format(1) #change to 15 minutes
z_sdo = df_sdo[df_sdo['WAVELNTH'] == 94]['GAEZ_OBS'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes_94 = df_sdo[df_sdo['WAVELNTH'] == 94]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes_131 = df_sdo[df_sdo['WAVELNTH'] == 131]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes_171 = df_sdo[df_sdo['WAVELNTH'] == 171]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes_193 = df_sdo[df_sdo['WAVELNTH'] == 193]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes_211 = df_sdo[df_sdo['WAVELNTH'] == 211]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes_304 = df_sdo[df_sdo['WAVELNTH'] == 304]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes_335 = df_sdo[df_sdo['WAVELNTH'] == 335]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()

cor_flux = []
for flux in [40,75,150,275,475]:
    # GOES (Select Telescope, Electron Flux and Parameter)
    df_goes = dd.read_parquet("/efs/sdoradbelt/data/goes_data/maged/goes_telescope{}_keV{}.parquet".format(2,flux))  
    cor_flux.append(df_goes['cor_flux'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate())


In [ ]:
# Graphing
fig, axs = plt.subplots(3,1,figsize=(30,10),gridspec_kw={'height_ratios': [1,1,1.5]})
fig.suptitle('Equator Data Selection',fontsize=27,y=0.935) #(October 15, 2019)

axs[0].plot(cor_flux[0],label='40'); axs[0].plot(cor_flux[1],label='75'); 
axs[0].plot(cor_flux[2],label='150'); axs[0].plot(cor_flux[3],label='275'); axs[0].plot(cor_flux[4],label='475')
axs[0].set_xticklabels([])
axs[0].set_ylabel('Electron Flux',fontsize=20)
axs[0].legend(loc="upper right",title='Energy (keV)',prop={'size': 14})
# units: e/(cm^2 s sr keV)

axs[1].plot(nspikes_94,label='94'); axs[1].plot(nspikes_131,label='131'); axs[1].plot(nspikes_171,label='171')
axs[1].plot(nspikes_193,label='193'); axs[1].plot(nspikes_211,label='211'); axs[1].plot(nspikes_304,label='304'); axs[1].plot(nspikes_335,label='335')
axs[1].set_xticklabels([])
#axs[0].set_ylim((0,1*10**6))
axs[1].set_ylabel('NSPIKES',fontsize=20)
axs[1].legend(loc="upper right",title=r'Wavelength ($\AA$)',prop={'size': 10})


axs[2].plot(z_sdo)
axs[2].axhline(y=-2*10**6,c='red',linewidth=0.5)
axs[2].axhline(y=2*10**6,c='red',linewidth=0.5)
axs[2].set_ylabel('SDO Latitude',fontsize=20)
axs[2].set_xlabel('Time',fontsize=20, labelpad=12)

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.savefig('Equator_Selection.png')
plt.show()

# High Energy Logarithm

In [ ]:
# Get headers in all wavelengths along with GOES-14 data. Convert to int32 for less memory. Select time span, cadence and interpolate in case of NaNs.
year_start = 2018; year_end = 2019
month_start = 1; month_end = 12
day_start = 1; day_end = 30
start_d = "{}-{}-{}".format(year_start,str(month_start).zfill(2),str(day_start).zfill(2)); end_d = "{}-{}-{}".format(year_end,str(month_end).zfill(2),str(day_end).zfill(2)) 
cadence = '{}min'.format(1) #change to 15 minutes
#[94,131,171,193,211,304,335]
z_sdo = df_sdo[df_sdo['WAVELNTH'] == 131]['GAEZ_OBS'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
nspikes_94 = df_sdo[df_sdo['WAVELNTH'] == 131]['NSPIKES'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
#[40,75,150,275,475]
df_goes = dd.read_parquet("/efs/sdoradbelt/data/goes_data/maged/goes_telescope{}_keV{}.parquet".format(2,475))  
cor_flux=df_goes['cor_flux'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()
df_goes2 = dd.read_parquet("/efs/sdoradbelt/data/goes_data/maged/goes_telescope{}_keV{}.parquet".format(2,40))  
cor_flux2=df_goes2['cor_flux'].astype(np.int32).loc[start_d:end_d].resample(cadence).mean().compute().interpolate()

In [ ]:
cor_flux[cor_flux<=0] = 1
cor_flux.plot()
plt.figure()
cor_flux_log = np.log(cor_flux)
cor_flux_log.plot()
print(stats.pearsonr(nspikes_94,cor_flux)[0])
print(stats.pearsonr(nspikes_94,cor_flux_log)[0])

# Rest Stuff - Notes

In [ ]:
print('Correlation Value for Corrected vs. Uncorrected')
print('r = ',stats.pearsonr(cor_flux,uncor_flux)[0])

In [ ]:
e_eq = []; nspikes_94_eq = []; lat_eq = []; 
for i in range(0,len(lat)):
    if lat[i] >= -2*10**6 and lat[i] <= 2*10**6:
        nspikes_94_eq.append(nspikes_94[i])
        e_eq.append(cor_flux[i])
        lat_eq.append(lat[i])

In [ ]:
norm_lat_eq = (lat_eq-np.min(lat_eq))/(np.max(lat_eq) - np.min(lat_eq))

In [ ]:
print('Number of total points: ',np.shape(cor_flux),np.shape(nspikes_94))
print('Number of points on equator: ',np.shape(e_eq),np.shape(nspikes_94_eq))

In [ ]:
plt.figure(figsize=(40,10))
plt.plot(e_eq)
plt.plot(norm_lat_eq*np.min(e_eq))

In [ ]:
plt.figure(figsize=(40,10))
plt.plot(nspikes_94_eq)
plt.plot(norm_lat_eq*np.min(nspikes_94_eq))

In [ ]:
print('Correlation Value for Spikes v Flux on Equator')
print('r = ',stats.pearsonr(nspikes_94_eq,e_eq)[0])